In [45]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ['GITHUB_APP_ID'] = os.getenv('GITHUB_APP_ID', '')
os.environ['GITHUB_APP_PRIVATE_KEY'] = os.getenv('GITHUB_APP_PRIVATE_KEY', '')
os.environ['GITHUB_REPOSITORY'] = os.getenv('GITHUB_REPOSITORY', '')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', '')

In [46]:
# Initialize tools first
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit

github_api = GitHubAPIWrapper()
toolkit = GitHubToolkit.from_github_api_wrapper(github_api)
github_tools = toolkit.get_tools()
# github_tools

In [47]:
from langchain_core.tools import tool
from typing import Dict

# Global variables to store GitHub tools
read_file_tool = None
overview_tool = None

def init_github_tools(github_tools):
    """Initialize GitHub tools from toolkit"""
    global read_file_tool, overview_tool
    
    read_file_tool = next((tool for tool in github_tools if tool.name == "Read File"), None)
    overview_tool = next((tool for tool in github_tools if tool.name == "Overview of existing files in Main branch"), None)
    
    if not read_file_tool or not overview_tool:
        raise ValueError("Required GitHub tools not found")

@tool
def read_readme_file() -> str:
    """
    Read the README.md file content from the repository.
    Returns the complete README content as a string.
    """
    if read_file_tool is None:
        return "Read File tool is not initialized. Please initialize GitHub tools first."
    try:
        readme_content = read_file_tool.invoke({ 
            "formatted_filepath": "README.md"
        })
        
        # Check if the result is an error message
        if "File not found" in readme_content or "404" in readme_content:
            return "README.md file not found in the repository"
        
        return readme_content
    except Exception as e:
        return f"Error reading README.md: {str(e)}"

@tool 
def get_project_overview() -> str:
    """
    Get an overview of all files in the main branch of the repository.
    Returns a structured view of the project's file organization with the project title.
    """
    if overview_tool is None:
        return "Overview tool is not initialized. Please initialize GitHub tools first."
    try:
        overview = overview_tool.invoke({})
        return overview + f"\n the project title is {os.getenv('GITHUB_REPOSITORY', 'Unknown Project')}"
    except Exception as e:
        return f"Error getting project overview: {str(e)}"

@tool
def detect_tech_stack() -> Dict[str, str] | str:
    """
    Read dependency files to detect technology stack.
    Returns a dictionary with file names as keys and their content as values.
    """
    tech_stack_files = {}
    
    # Common dependency files to check
    dependency_files = [
        "package.json",
        "requirements.txt", 
        "Cargo.toml",
        "pom.xml",
        "build.gradle",
        "go.mod",
        "composer.json",
        "Gemfile",
        "poetry.lock",
        "yarn.lock",
        "Pipfile"
    ]
    if read_file_tool is None:
        return "Read File tool is not initialized. Please initialize GitHub tools first."
    
    for file_name in dependency_files:
        try:
            content = read_file_tool.invoke({
                "formatted_filepath": file_name
            })
            
            # Check if the result is an error message
            if "File not found" in content or "404" in content:
                # Skip files that don't exist
                continue
                
            tech_stack_files[file_name] = content
        except Exception:
            # Skip if there's any other error
            continue
    
    return tech_stack_files 

# Tools list for LangGraph
tools = [detect_tech_stack, read_readme_file, get_project_overview]

In [48]:

# Initialize the custom tools
init_github_tools(github_tools)

# Now use the tools
readme_content = read_readme_file.invoke({})
overview = get_project_overview.invoke({})
tech_files = detect_tech_stack.invoke({})

print("Tech stack files found:", list(tech_files.keys()))

Tech stack files found: ['requirements.txt']


In [49]:
from langchain_google_genai import ChatGoogleGenerativeAI
MODEL_NAME = "gemini-2.0-flash"
model = ChatGoogleGenerativeAI(model=MODEL_NAME)
model

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x772d5a791d00>, default_metadata=(), model_kwargs={})

In [50]:
# Step 1: Data Analysis Prompt
ANALYSIS_PROMPT = """You are a project analyzer specializing in student GitHub projects.

## AVAILABLE TOOLS:
- read_readme_file(): Get README content
- get_project_overview(): Get file structure overview  
- detect_tech_stack(): Get technology stack from dependency files

## YOUR TASK:
Gather and analyze comprehensive project information:

1. **Use ALL available tools** to collect project data
2. **Analyze the project** for:
   - Technical complexity and sophistication
   - Key technologies and implementation challenges
   - Learning outcomes demonstrated
   - Real-world problem solved
   - Development skills showcased

## OUTPUT FORMAT:
Provide a structured analysis in this format:

**PROJECT OVERVIEW:**
- Name: [project name]
- Purpose: [what it does and why built]
- Complexity Level: [beginner/intermediate/advanced]

**TECHNICAL STACK:**
- Primary Technologies: [list main techs]
- Key Dependencies: [important libraries/frameworks]
- Architecture Pattern: [if applicable]

**LEARNING HIGHLIGHTS:**
- Skills Demonstrated: [specific technical skills shown]
- Challenges Overcome: [implementation challenges]
- Growth Areas: [what was learned/improved]

**KEY FEATURES:**
- [List 3-5 main functionalities]

**SIGNIFICANCE:**
- Why this project matters for career development
- What it demonstrates to potential employers

Begin by using the tools to gather project information, then provide your analysis."""

# Step 2: Content Strategy Prompt
STRATEGY_PROMPT = """You are a LinkedIn content strategist specializing in student developer content.

## CONTEXT:
You have received a project analysis. Now create a content strategy for a LinkedIn post.

## YOUR TASK:
Create a strategic approach for showcasing this project on LinkedIn.

## STRATEGY REQUIREMENTS:

**TARGET AUDIENCE:**
- Hiring managers and recruiters
- Fellow students and developers  
- Tech professionals and mentors

**CONTENT STRATEGY:**
1. **Hook Strategy**: What angle will grab attention in first 125 characters?
2. **Key Selling Points**: Top 3 technical/learning highlights to emphasize
3. **Narrative Arc**: How to tell the learning journey story
4. **Engagement Angle**: What question or CTA will drive comments?
5. **Hashtag Strategy**: 5-8 strategic hashtags for maximum reach

## OUTPUT FORMAT:
**HOOK ANGLE:** [Your chosen attention-grabbing approach]

**KEY SELLING POINTS:**
1. [First main highlight]
2. [Second main highlight]  
3. [Third main highlight]

**NARRATIVE STRUCTURE:**
- Opening: [How to introduce the project]
- Body: [How to present technical details]
- Learning: [How to showcase growth]
- Closing: [How to end with engagement]

**ENGAGEMENT STRATEGY:**
- Question: [Specific question to ask readers]
- CTA: [Call-to-action for engagement]

**HASHTAGS:** [5-8 strategic hashtags]

**TONE NOTES:** [Any specific tone considerations]

Provide a clear, actionable content strategy."""

# Step 3: Writing Prompt
WRITING_PROMPT = """You are a professional LinkedIn writer specializing in student developer content.

## CONTEXT:
You have received:
1. Project analysis with technical details
2. Content strategy with narrative approach

## YOUR TASK:
Write a compelling LinkedIn post following the strategy provided.

## WRITING REQUIREMENTS:

**STRUCTURE** (Follow the strategy provided):
1. **Hook** (1-2 lines, first 125 characters crucial)
2. **Project Introduction** (2-3 lines)  
3. **Technical Highlights** (3-4 lines)
4. **Key Features** (2-3 bullet points with emojis)
5. **Learning Journey** (1-2 lines)
6. **Call-to-Action** (1-2 lines)
7. **Repository Link**: "👉 Check it out: [github_url]"
8. **Hashtags** (use strategy provided)

**FORMATTING:**
- Target length: 1300-1600 characters
- Use line breaks for mobile readability
- Bold key technical terms
- Use 2-3 strategic emojis max
- Bullet points with • (LinkedIn compatible)

**TONE:**
- First person, authentic student voice
- Enthusiastic but humble
- Growth-focused and future-oriented
- Professional yet approachable

## OUTPUT:
Provide ONLY the final LinkedIn post content, properly formatted and ready to publish.

Write the post now following all requirements."""

# Step 4: Review Prompt
REVIEW_PROMPT = """You are a LinkedIn content quality reviewer specializing in student developer posts.

## YOUR TASK:
Review the LinkedIn post and ensure it meets all quality standards.

## QUALITY CHECKLIST:
✓ Hook creates curiosity within first 125 characters?
✓ Character count between 1300-1600?
✓ Shows technical competence without being intimidating?
✓ Demonstrates learning growth and development mindset?
✓ Appeals to hiring managers showing employability?
✓ Engages fellow students and developers?
✓ Clear call-to-action for engagement?
✓ Mobile-friendly formatting with proper spacing?
✓ Authentic student voice while maintaining professionalism?
✓ Proper hashtag strategy (5-8 relevant tags)?
✓ Strategic emoji use (2-3 max)?

## OUTPUT FORMAT:
**QUALITY SCORE:** [X/10]

**REVIEW FEEDBACK:**
- ✅ Strengths: [What works well]
- ⚠️ Issues: [What needs improvement]
- 🔧 Suggestions: [Specific improvements]

**REVISED POST:** [Only if changes needed, provide improved version]

If score is 8+ and no major issues, approve the post as-is.
If score is below 8, provide specific revisions."""

In [52]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from typing_extensions import TypedDict
from typing import Annotated
import operator


# State definition
class LinkedInPostState(TypedDict):
    github_url: str
    project_analysis: str
    content_strategy: str
    draft_post: str
    review_feedback: str
    final_post: str
    messages: Annotated[list, operator.add]


analysis_agent = create_react_agent(
    model=model,
    tools=tools,  # Your GitHub tools
    prompt=ANALYSIS_PROMPT,
)

strategy_agent = create_react_agent(
    model=model,
    tools=[],  # No tools needed for strategy
    prompt=STRATEGY_PROMPT,
)

writing_agent = create_react_agent(
    model=model,
    tools=[],  # No tools needed for writing
    prompt=WRITING_PROMPT,
)

review_agent = create_react_agent(
    model=model,
    tools=[],  # No tools needed for review
    prompt=REVIEW_PROMPT,
)


# Node functions
def analysis_node(state: LinkedInPostState):
    """Analyze the GitHub project"""
    result = analysis_agent.invoke(
        {"messages": [("human", f"Analyze this GitHub project: {state['github_url']}")]}
    )

    return {
        "project_analysis": result["messages"][-1].content,
        "messages": [("system", "Project analysis completed")],
    }


def strategy_node(state: LinkedInPostState):
    """Create content strategy"""
    context = f"Project Analysis:\n{state['project_analysis']}"

    result = strategy_agent.invoke(
        {
            "messages": [
                ("human", f"{context}\n\nCreate a content strategy for this project.")
            ]
        }
    )

    return {
        "content_strategy": result["messages"][-1].content,
        "messages": [("system", "Content strategy created")],
    }


def writing_node(state: LinkedInPostState):
    """Write the LinkedIn post"""
    context = f"""Project Analysis:
{state['project_analysis']}

Content Strategy:
{state['content_strategy']}

GitHub URL: {state['github_url']}"""

    result = writing_agent.invoke(
        {
            "messages": [
                (
                    "human",
                    f"{context}\n\nWrite the LinkedIn post following the strategy.",
                )
            ]
        }
    )

    return {
        "draft_post": result["messages"][-1].content,
        "messages": [("system", "Draft post created")],
    }


def review_node(state: LinkedInPostState):
    """Review and finalize the post"""
    result = review_agent.invoke(
        {
            "messages": [
                ("human", f"Review this LinkedIn post:\n\n{state['draft_post']}")
            ]
        }
    )

    review_content = result["messages"][-1].content

    # Check if revision is needed (simple logic)
    if "REVISED POST:" in review_content:
        # Extract revised post
        final_post = review_content.split("REVISED POST:")[-1].strip()
    else:
        # Use original draft
        final_post = state["draft_post"]

    return {
        "review_feedback": review_content,
        "final_post": final_post,
        "messages": [("system", "Post review completed")],
    }


# Build the workflow
workflow = StateGraph(LinkedInPostState)

# Add nodes
workflow.add_node("analyze", analysis_node)
workflow.add_node("strategize", strategy_node)
workflow.add_node("write", writing_node)
workflow.add_node("review", review_node)

# Add edges (linear flow)
workflow.add_edge(START, "analyze")
workflow.add_edge("analyze", "strategize")
workflow.add_edge("strategize", "write")
workflow.add_edge("write", "review")
workflow.add_edge("review", END)

# Compile the workflow
linkedin_agent = workflow.compile()


# Usage example
def generate_linkedin_post(github_url: str):
    """Generate a LinkedIn post from a GitHub repository"""

    initial_state = {
        "github_url": github_url,
        "project_analysis": "",
        "content_strategy": "",
        "draft_post": "",
        "review_feedback": "",
        "final_post": "",
        "messages": [],
    }

    # Run the workflow
    final_state = linkedin_agent.invoke(initial_state)

    return final_state


# Example usage:
result = generate_linkedin_post(
    "https://github.com/BENHIMA-Mohamed-Amine?tab=repositories"
)
print(result["final_post"])

**

Struggling to assess coding skills effectively? 🤯 I built an AI-powered service that automatically generates and evaluates code exercises using LLMs! 👩‍💻

I'm excited to share my latest project: an agentic service that creates QCMs (multiple choice questions) and coding exercises, and automatically evaluates code submissions using LLM-powered agents. All monitored with LangSmith for debugging and continuous improvement!

Technically, this project leverages **Python**, **Flask**, **LangChain**, and the **LangGraph React Agent** for workflow orchestration. I also used the **LangChain Sandbox Tool** for secure code execution and **LangSmith** for monitoring and debugging.

Key features include:
*   Generates QCMs (multiple choice questions) 📝
*   Generates coding exercises 💻
*   Automatically evaluates user-submitted code using LLM agents ✅

Throughout this project, I learned a ton about agentic workflow design and secure code execution. One key challenge was implementing safe code ex

In [40]:
LINKEDIN_POST_PROMPT = """You are a professional LinkedIn content creator specializing in showcasing student technical projects for career development and skill demonstration.

Your task is to create a compelling LinkedIn post that showcases a GitHub project to demonstrate technical skills, learning journey, and potential to hiring managers, fellow students, developers, and tech enthusiasts.

## AVAILABLE TOOLS:
- read_readme_file(): Get README content
- get_project_overview(): Get file structure overview  
- detect_tech_stack(): Get technology stack from dependency files

## CONTEXT GATHERING:
1. First, gather comprehensive information about the project:
   - Repository name and description
   - Technology stack and dependencies
   - Key features from README
   - Project structure and organization
   - Recent development activity
   - Learning challenges overcome
   - Implementation complexity level

## LINKEDIN POST REQUIREMENTS:

### STUDENT-FOCUSED STRUCTURE:
1. **Hook** (1-2 lines, first 125 characters crucial): 
   - Engaging question or bold statement about learning/building
   - Focus on curiosity, growth, or problem-solving passion
   - Use "you" to address readers directly

2. **Project Introduction** (2-3 lines):
   - What the project does and why you built it
   - Emphasize learning motivation or real-world problem solved
   - Include specific metrics when available

3. **Technical Highlights** (3-4 lines):
   - Key technologies learned/used
   - Interesting implementation challenges overcome
   - Growth demonstrated through the project
   - Focus on skill development narrative

4. **Key Features** (2-3 lines with bullet points):
   - Main functionalities using emojis for visual appeal
   - Highlight technical complexity appropriate for skill level
   - Show understanding of software engineering principles

5. **Learning Journey** (1-2 lines):
   - Personal growth aspect or "aha moment"
   - Challenges faced and how you overcame them
   - Skills acquired or strengthened

6. **Call-to-Action** (1-2 lines):
   - Specific engagement request and feedback
   - Invite collaboration or knowledge sharing

7. **Repository Link**: "👉 Check it out: [github_url]"

8. **Hashtags** (5-8 strategic tags)

### LINKEDIN ALGORITHM OPTIMIZATION:
- Target length: 1300-1600 characters for maximum reach
- First 2 lines must hook readers (mobile preview cutoff)
- Include question to increase comments and engagement
- Use line breaks and strategic formatting for mobile scanning
- Write for mixed audience (technical and non-technical)

### STUDENT TONE & STYLE:
- **Enthusiastic but humble**: Show passion without overselling
- **Growth-focused**: Emphasize learning journey and development
- **Authentic voice**: Genuine excitement about technology
- **Future-oriented**: Hint at career aspirations and continued learning
- **Community-minded**: Seeking guidance, collaboration, and connection
- Write in first person with confidence but acknowledge learning process

### FORMATTING FOR ENGAGEMENT:
- Use line breaks for mobile readability
- Bold key technical terms or achievements
- Strategic emoji use (2-3 max, focused on tech/learning themes)
- Bullet points for features (• not - for LinkedIn compatibility)
- Write in format compatible with LinkedIn's text editor

### CONTENT GUIDELINES FOR STUDENTS:
- **Problem-Solution Focus**: Show you understand real-world applications
- **Technical Depth**: Demonstrate coding skills without overwhelming non-technical readers
- **Learning Narrative**: Include what you learned or skill gaps filled
- **Professional Growth**: Connect project to career development goals
- **Industry Relevance**: Show awareness of current tech trends and practices
- **Collaboration Ready**: Indicate openness to feedback and team opportunities

### HASHTAG STRATEGY:
- **Technology Stack**: 2-3 specific tech tags (#Python, #React, #Node.js)
- **Development**: 2-3 broader dev tags (#WebDevelopment, #FullStack, #Backend)
- **Community**: 1-2 community tags (#OpenSource, #GitHub, #BuildingInPublic)
- **Student/Career**: 1-2 growth tags (#LearningInPublic, #StudentDeveloper, #TechSkills)
- **Industry**: 1 relevant industry tag if applicable (#AI, #WebDev, #MobileApp)

### QUALITY CHECKLIST:
✓ Hook creates curiosity within first 125 characters?
✓ Shows technical competence without being intimidating?
✓ Demonstrates learning growth and development mindset?
✓ Appeals to hiring managers showing employability?
✓ Engages fellow students and developers for community building?
✓ Clear next step for interested readers?
✓ Mobile-friendly formatting with proper spacing?
✓ Authentic student voice while maintaining professionalism?

### STUDENT SUCCESS METRICS TO HIGHLIGHT:
- Lines of code or project complexity
- Technologies learned or mastered
- Problems solved or challenges overcome
- Time invested or project duration
- Features implemented or functionality achieved
- Performance improvements or optimizations made

## INSTRUCTIONS:
1. **Information Gathering**: Use ALL available tools to gather comprehensive project information
2. **Student Context Analysis**: Analyze how the project demonstrates growth, learning, and technical skills
3. **Audience Adaptation**: Consider mixed audience of students, developers, and potential employers
4. **LinkedIn Optimization**: Apply algorithm-friendly formatting and engagement strategies
5. **Post Creation**: Generate a post that showcases technical skills while telling a compelling learning story
6. **Professional Positioning**: Frame the project to enhance career prospects while staying authentic to student experience

Begin by gathering all project information using the available tools, then create an engaging LinkedIn post that positions you as a growing developer with strong technical potential and genuine passion for learning.
"""

In [41]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()

# control iteration loop
max_iterations = 7
recursion_limit = 2 * max_iterations + 1

agent = create_react_agent(
    model=model,
    tools=tools,
    prompt=LINKEDIN_POST_PROMPT,
    checkpointer=memory,
    debug=True,
).with_config(recursion_limit=recursion_limit)

In [42]:
config = {"configurable": {"thread_id": "linkedin-agent-1"}}

In [43]:
response = agent.invoke(
    {"messages": [("user", "Create a professional LinkedIn post showcasing this GitHub repository")]}, 
    config # type: ignore
)

[values] {'messages': [HumanMessage(content='Create a professional LinkedIn post showcasing this GitHub repository', additional_kwargs={}, response_metadata={}, id='ace4d3a3-f87b-471d-a0a8-5103ba608c39')]}
[updates] {'agent': {'messages': [AIMessage(content="Okay, I'm ready to create a LinkedIn post. First, I need to gather information about the project using the available tools. I'll start by reading the README file, getting the project overview, and detecting the tech stack.", additional_kwargs={'function_call': {'name': 'detect_tech_stack', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--0a45aef2-fd4e-4b31-954c-de3e7a406212-0', tool_calls=[{'name': 'read_readme_file', 'args': {}, 'id': '98d897e1-6e39-44ad-86f7-6ce407faaf80', 'type': 'tool_call'}, {'name': 'get_project_overview', 'args': {}, 'id': '5b385576-8a36-48bc-9757-6bae491892f9', 'typ

In [44]:
print(response["messages"][-1].content)

Are you fascinated by the power of AI agents? 🤖 I built a project to generate coding exercises and automatically evaluate code using LangChain!

This project leverages LLM-powered agents to create QCMs (multiple choice questions) and coding challenges, then evaluates user-submitted code. It was driven by my interest in agentic workflows and applying them to coding education. I'm excited to share what I've learned!

Here are some technical highlights:
*   Utilized LangChain and LangGraph to orchestrate complex agent interactions.
*   Implemented a Flask API to serve the agentic coding exercise service.
*   Integrated LangChain Sandbox for secure code execution.

Key Features:
*   ✅ Generates QCMs and coding exercises.
*   ✅ Automatically evaluates user-submitted code.
*   ✅ Uses LangSmith for monitoring and debugging.

One of the biggest challenges was understanding the intricacies of LangGraph and how to effectively manage multi-step workflows. Overcoming this hurdle significantly deep